In [57]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/Vegetable_Images_2'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras import models, layers, optimizers  # Add this import statement

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [59]:
# Constants
IMAGE_SIZE = 224
BATCH_SIZE = 64  # You can adjust this based on your GPU memory
CHANNELS = 3
EPOCHS = 10
CLASS_NAMES = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot',
               'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

In [60]:
# Load pre-trained MobileNetV2 model (include_top=False excludes the final dense layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

In [61]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [62]:
# Model Architecture
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(CLASS_NAMES), activation='softmax'),
])

In [63]:
# Compile Model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),  # Adjust learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model Summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_5  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_10 (Dense)            (None, 64)                81984     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 15)                975       
                                                                 
Total params: 2340943 (8.93 MB)
Trainable params: 8295

In [64]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Data Augmentation
train_data_augmentation = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [65]:
# Load and preprocess data
train_generator = train_data_augmentation.flow_from_directory(
    '/content/drive/MyDrive/Vegetable_Images_2/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES
)

Found 15020 images belonging to 15 classes.


In [66]:
# Assuming you have a directory for validation data similar to the training data
val_folder_path = '/content/drive/MyDrive/Vegetable_Images_2/validation'

# Validation Data Augmentation (you can adjust this based on your needs)
val_data_augmentation = ImageDataGenerator(rescale=1./255)

# Create a validation data generator
val_generator = val_data_augmentation.flow_from_directory(
    val_folder_path,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # Assuming you have multiple classes
)

Found 3000 images belonging to 15 classes.


In [67]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10
235/235 [==============================] - 1018s 4s/step - loss: 1.9314 - accuracy: 0.4001 - val_loss: 0.7224 - val_accuracy: 0.9220 - lr: 1.0000e-04
Epoch 2/10
235/235 [==============================] - 1016s 4s/step - loss: 0.8765 - accuracy: 0.7435 - val_loss: 0.2692 - val_accuracy: 0.9690 - lr: 1.0000e-04
Epoch 3/10
235/235 [==============================] - 1028s 4s/step - loss: 0.5456 - accuracy: 0.8494 - val_loss: 0.1490 - val_accuracy: 0.9773 - lr: 1.0000e-04
Epoch 4/10
235/235 [==============================] - 1000s 4s/step - loss: 0.4093 - accuracy: 0.8850 - val_loss: 0.1020 - val_accuracy: 0.9807 - lr: 1.0000e-04
Epoch 5/10
235/235 [==============================] - 1020s 4s/step - loss: 0.3338 - accuracy: 0.9079 - val_loss: 0.0777 - val_accuracy: 0.9827 - lr: 1.0000e-04
Epoch 6/10
235/235 [==============================] - 1028s 4s/step - loss: 0.2913 - accuracy: 0.9206 - val_loss: 0.0636 - val_accuracy: 0.9847 - lr: 1.0000e-04
Epoch 7/10
235/235 [==============

In [ ]:
# Save the model
model.save('/content/drive/MyDrive/Vegetable_Images_2/vegetable_model.h5')

# Evaluate the model on the test set
test_data_augmentation = ImageDataGenerator(rescale=1./255)

test_generator = test_data_augmentation.flow_from_directory(
    '/content/drive/MyDrive/Vegetable_Images_2/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES
)

test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


Found 3000 images belonging to 15 classes.
47/47 [==============================] - 134s 3s/step - loss: 0.0550 - accuracy: 0.9870
Test Accuracy: 98.70%
